In [1]:
# Работа с массивами данных
import numpy as np 

# Работа с таблицами
import pandas as pd

# Отрисовка графиков
import matplotlib.pyplot as plt

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential,load_model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation 

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Заполнение последовательностей до определенной длины
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Матрица ошибок классификатора
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Кодирование тестовых меток
from sklearn.preprocessing import LabelEncoder

# Разбиение на тренировочную и тестовую выборки
from sklearn.model_selection import train_test_split

# Загрузка датасетов из облака google
import gdown

# Отрисовка графиков
import matplotlib.pyplot as plt

%matplotlib inline
import gc
import os
import random

In [2]:
# Загрузка данных из файла в таблицу
df = pd.read_csv('train_dataset_train.csv')

In [10]:
df.head(20)

,id,name,groups
0,1034,шок цена пена д душа бритья куппер мл акс,10
1,1035,мин вода нагутская № л,0
2,1036,пельмени домашние вес по прямицино,10
3,1037,печенье овсяно гречневое эко бот,10
4,1038,спред растительно жировой масляничка г с змж,10
5,1039,пельмени золото дэмк,10
6,1040,мдб колб доктор в с б о в у,10
7,1041,щипцы кухон см пласт нерж сталь ручка vetta,10
8,1042,игр вино просекко националь б,10
9,1043,лапша удон курица с овощами соус терияки,1


In [4]:
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

In [5]:
df = df.applymap(lambda s: s.replace('"','') if type(s) == str else s)

In [6]:
from string import punctuation  # Модуль для работы со строками

# Удаление символов пунктуации
def remove_punctuation(my_string):
    return "".join([ch if ch not in punctuation else ' ' for ch in my_string]) # Замена на пробелы!!!

# Удаление чисел
def remove_numbers(my_string):
    return ''.join([i if not i.isdigit() else ' ' for i in my_string])

# Замена множественных пробелов на одинарный пробел
import re
def remove_multiple_spaces(my_string):
	return re.sub(r'\s+', ' ', my_string, flags=re.I)

# Обработка строк + дополнительно приведение к нижнему регистру
preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower().strip())))) 

# Создаем колонку в датасете для обработанного текста
df['name'] = list(map(preproccessing, df['name']))
df.head()

,id,name,groups
0,1034,шок цена пена д душа бритья куппер мл акс,10
1,1035,мин вода нагутская № л,0
2,1036,пельмени домашние вес по прямицино,10
3,1037,печенье овсяно гречневое эко бот,10
4,1038,спред растительно жировой масляничка г с змж,10


# генерация одной процедурой

In [7]:
def give_please_new_fantastik_dataset(n,COUNT=1000):  #n - по какой группе товаров
    #САМОЕ СЕРДЦЕ: БЕРЕМ СЛУЧАЙНОЕ СЛОВО ИЗ СЛОВАРЯ, ПОТОМ ВЫБИРАЕМ ЗА НИМ СЛЕДУЮЩЕЕ И ТАК В РЕКУРСИИ
    def get_variant(fraza, dlina = 5):
        if len(fraza)+1 == dlina:
            return fraza
        key =fraza[-1] 
        var = big_dict[key] #получили словарь слов, следующих за нашим из словаря надо выбрать случайный ключ. 
        candidat = [x[0] for x in var] #cсписок слов
        weights = [x[1] for x in var] #их частоты
        if not candidat: #нет кандидатов на продолжение
            return fraza
        #candidat =candidat[:15]  #10 САМЫХ УСПЕШНЫХ КАНДИДАТОВ
        
        new_word = fraza[-1] #Берем последнее слово из фразы, чтобы подобрать новое, которого нет еще во фразе
        popit=0 #это чтобы не закрутиться в цикле
        while (new_word in fraza):
            #print(new_word, fraza)
            new_word = random.choices(candidat, weights = weights )[0]
            popit +=1
            if new_word == 'END' or popit>20: #досрочное окончание
                return fraza


#         if new_word == 'END': #досрочное окончание
#             return fraza
        fraza.append(new_word)
        return get_variant(fraza,dlina)
#=================================================================================================
    
    
    
    
    
    df_cl   = df.loc[df['groups']==n] 
    text_for = df_cl.name.tolist()
    #добавляем слово END В конце каждого предложения. 
    text_for = [x+' END' for x in text_for]  
    #удаляем однобуквенные
    text_for = [ ' '.join([i for i in x.split(' ') if len(i)>1 ]) for x in text_for]  
    
    
    
    
    big_text = ' '.join(text_for)
    big_text =  big_text.split(' ')
    #print(big_text[:100])
    black_list =[]  #можно добавить слова, которые убрать из формируемого датасета
    big_dict = {big_text[0]:{}}  #у нас будет словарь словарей. Ключ словара - слово. Значение - другие слова, которые идут за ним и количество повторов этих слов
    pred = big_text[0]
    for i in range(1, len(big_text)):
        tek = big_text[i]
        if tek == '':
            continue
        if tek in black_list:
            continue
        if tek in big_dict[pred]: #если это слово уже было в списке у предыдущего слова, то мы там просто увеличим на 1 частоту, если не было то создадим
            big_dict[pred][tek] = big_dict[pred][tek] + 1
        else:
            big_dict[pred][tek] = 1

        #если текущего слова еще не было, то мы его просто добавим в словарь
        if tek not in big_dict:
            big_dict[tek] = {}
        pred = tek
    #словарь сделали, теперь отсортируем словари по каждому слову
    for key, value in big_dict.items():
        value = sorted(value.items(), key=lambda x: -x[1])
        value = [x for x  in value if x[1]>=3] #не будем брать низко частотные слова
        big_dict[key] = value
        
    #генерируем по словарю    
    ext = []
    empt = 0
    i =0
    while len(ext)<COUNT:
        a =[]
        a.append('END')
        
        i +=1
        #print(i)
        new =  get_variant(a,10)

        new =  new[1:]
        new = ' '.join(new)
        if not new in ext:
            ext.append(new)
            empt = 0
        else:
            empt += 1
        if empt>COUNT:
            print('данные перестали генерироваться. Сгенерили ', len(ext))
    
    
    
    new = pd.DataFrame()
    gr =[str(n)]*len(ext)
    new['id'] = gr
    new['name'] = ext
    new['groups'] = gr
    out_file = f'ext{COUNT}_{n}.csv'
    new.to_csv(out_file, sep=',', encoding ='utf-8', index = False)
    print('сохранено в ',out_file)

In [8]:
give_please_new_fantastik_dataset(2,3000)

сохранено в  ext3000_2.csv


# пробуем генерировать датасет

In [14]:
df_cl   = df.loc[df['groups']==2] 

In [15]:
text_for = df_cl.name.tolist()

In [16]:
#добавляем слово END В конце каждого предложения. 
text_for = [x+' END' for x in text_for]  
#удаляем однобуквенные
text_for = [ ' '.join([i for i in x.split(' ') if len(i)>1 ]) for x in text_for]  


In [15]:
text_for[:10]

['кофе топленое молоко печенье 5кг дымка (киров) кг END',
 'молоко топленое починки 3,5% END',
 '31: 3376242 рогач.мол.сгущ.8,5% ж/б 380г END',
 'печенье правильный выбор топленое молоко 1/500 END',
 'десерт гл сгущ 23% пленка 0,04кг миньоны END',
 'молоко питьевое латона 2,5% топленое END',
 'печенье сахар сгущенное молоко 430г пач(донской END',
 'напиток молоко шоколадом 250 мл END',
 'печенье диарит батончики со вкусом сгущ молоко кг (кор (3 кг)) END',
 'молоко первозданное топленое 3.5% 0.750мл END']

In [17]:
big_text = ' '.join(text_for)
big_text =  big_text.split(' ')
print(big_text[:100])
#black_list =['печенье','конфеты','творог','круассан','сырок','творог','вафли','печ']
black_list =[]
big_dict = {big_text[0]:{}}  #у нас будет словарь словарей. Ключ словара - слово. Значение - другие слова, которые идут за ним и количество повторов этих слов
pred = big_text[0]
for i in range(1, len(big_text)):
    tek = big_text[i]
    if tek == '':
        continue
    if tek in black_list:
        continue
    if tek in big_dict[pred]: #если это слово уже было в списке у предыдущего слова, то мы там просто увеличим на 1 частоту, если не было то создадим
        big_dict[pred][tek] = big_dict[pred][tek] + 1
    else:
        big_dict[pred][tek] = 1
        
    #если текущего слова еще не было, то мы его просто добавим в словарь
    if tek not in big_dict:
        big_dict[tek] = {}
    pred = tek
#словарь сделали, теперь отсортируем словари по каждому слову
for key, value in big_dict.items():
    value = sorted(value.items(), key=lambda x: -x[1])
    value = [x for x  in value if x[1]>=3] #не будем брать низко частотные слова
    big_dict[key] = value

['кофе', 'топленое', 'молоко', 'печенье', 'кг', 'дымка', 'киров', 'кг', 'END', 'молоко', 'топленое', 'починки', 'END', 'рогач', 'мол', 'сгущ', 'END', 'печенье', 'правильный', 'выбор', 'топленое', 'молоко', 'END', 'десерт', 'гл', 'сгущ', 'пленка', 'кг', 'миньоны', 'END', 'молоко', 'питьевое', 'латона', 'топленое', 'END', 'печенье', 'сахар', 'сгущенное', 'молоко', 'пач', 'донской', 'END', 'напиток', 'молоко', 'шоколадом', 'мл', 'END', 'печенье', 'диарит', 'батончики', 'со', 'вкусом', 'сгущ', 'молоко', 'кг', 'кор', 'кг', 'END', 'молоко', 'первозданное', 'топленое', 'мл', 'END', 'прод', 'молокосод', 'сгущ', 'белый', 'медведь', 'бут', 'END', 'молоко', 'топленое', 'гр', 'вкусняев', 'пэт', 'END', 'вафли', 'рф', 'коровка', 'топленое', 'молоко', 'END', 'печенье', 'сдобное', 'орешки', 'со', 'сгущ', 'молоком', 'кг', 'каменск', 'уральский', 'END', 'сливки', 'сгущ', 'волоконовское', 'ведро', 'END', 'мол', 'пр', 'сырок']


In [25]:
#big_dict - содердит словарь. В нем ключ - слово, значение - список слов, следующих за ключом с указанием количества повторов
big_dict['молоко']

[('топленое', 446),
 ('END', 276),
 ('ультрапастеризованное', 159),
 ('питьевое', 121),
 ('кг', 70),
 ('гр', 61),
 ('фермерское', 41),
 ('топл', 38),
 ('вес', 38),
 ('бзмж', 27),
 ('сливочно', 24),
 ('топлёное', 23),
 ('село', 23),
 ('творог', 20),
 ('паст', 19),
 ('простоквашино', 18),
 ('печенье', 15),
 ('parmalat', 15),
 ('сахарное', 12),
 ('зауралья', 12),
 ('домик', 11),
 ('топлен', 10),
 ('рот', 10),
 ('топ', 9),
 ('дымка', 9),
 ('пастер', 9),
 ('вятушка', 9),
 ('мл', 9),
 ('любятово', 9),
 ('эконом', 9),
 ('слив', 9),
 ('пит', 9),
 ('уп', 8),
 ('шт', 8),
 ('гост', 7),
 ('фрау', 7),
 ('фп', 7),
 ('воронеж', 7),
 ('пакет', 7),
 ('вкуснотеево', 7),
 ('добрая', 6),
 ('клевер', 6),
 ('молочный', 6),
 ('восстановленное', 6),
 ('тм', 6),
 ('амурское', 6),
 ('мумуня', 6),
 ('белый', 6),
 ('глазир', 6),
 ('городец', 6),
 ('на', 6),
 ('безлактозное', 6),
 ('пестравка', 6),
 ('обезжиренный', 5),
 ('из', 5),
 ('цельное', 5),
 ('свитлогорье', 5),
 ('глаз', 5),
 ('гринагро', 5),
 ('россия', 5

In [29]:
#САМОЕ СЕРДЦЕ: БЕРЕМ СЛУЧАЙНОЕ СЛОВО ИЗ СЛОВАРЯ, ПОТОМ ВЫБИРАЕМ ЗА НИМ СЛЕДУЮЩЕЕ И ТАК В РЕКУРСИИ
def get_variant(fraza, dlina = 5):
    if len(fraza)+1 == dlina:
        return fraza
    key =fraza[-1] 
    var = big_dict[key] #получили словарь слов, следующих за нашим из словаря надо выбрать случайный ключ. Предлагаю брать не те у которых значение 1.
    candidat = [x[0] for x in var]
    if not candidat: #нет кандидатов на продолжение
        return fraza
    candidat =candidat[:15]  #10 САМЫХ УСПЕШНЫХ КАНДИДАТОВ
    new_word = ''
#     while (not (new_word in fraza) and new_word):
    n = list(set(candidat) - set(fraza))
    if len(n) == 0: #слов для выбора нет, 
        return fraza
    new_word = random.choice(list(n))
        
        
    if new_word == 'END': #досрочное окончание
        return fraza
    fraza.append(new_word)
    return get_variant(fraza,dlina)


In [30]:
ext = []
COUNT = 9000
for k in range(COUNT):
    a =[]
    a.append('END')
    new =  get_variant(a,10)
    
    new =  new[1:]
    new = ' '.join(new)
    ext.append(new)
    print(new)

трубочка ваф.со
конфеты коробке птичье молоко простоквашино топленое 4,0% 930г
молоко ультрапастеризованное 1,5% 950мл бзмж 2,5% 1000г
топленое бзмж 4% пэт 930г
молоко топленое 4% 0,9л пэт 930 гр
печенье сдобное орешки вареной сгущ. молоком,
пряники топленое село зеленое ультрапастеризованное 3,2% простоквашино 1,5%
молоко топленое 0,5л ф/п
конфеты птичье молоко 1кг (вес)
сырок твор
молоко 5% 180г флоупак/10/бзмж
печенье аромат топленое пэт 930мл
круассан вар.сгущ.95г
пирожное мирэль эклеры со сливками
сырок гл пингв понго
сыр черный герцог 50% топленое эго 3,2% 1/12
пряник топленое 2,5% 900 мл.
сырок глазир вар сгущ20%40г
трубочка молоком сгущ мол гост 8,5% 360г ж/б
топленое эго 3,2% 1л пл/б
трубочка вафельная
печенье топлен молоко простоквашино ультрапастеризованное 4% 0,5л т/б
круассан вар сгущ20%40г
пряник тульский вар.сгущ.140г
круассан со сливочным
круассан вар сгущ молоком сгущ. молока ультрапастеризованноелазовское
напиток молоко 50% топленое 3,5% 1л бутылка бзмж
вафли трубочки

вафли трубочки со сгущ. молока ультрапастеризованноелазовское
сырок ваф. рожке со сгущ.,
бзмж суфле
творог 9% 180г флоупак/10/бзмж
печенье сахар сгущенное молоко фермерское подворье ультрапастеризованное 3,2%
пряник тульский вар.сгущ.140
топленое бзмж сырок гл пингв понго
конфеты коробке птичье молоко село зеленое 3,2% 900мл
пряник топленое 4% пэт бут.
сыр глаз вар сгущ 16% 40г премиум мол.сгущ
бзмж 1л ультрапастеризованное 3.2%
бзмж творог просто молоко простоквашино сырок глазир вар
топленое вологжанка 4% 1л бзмж молоко фермерское подворье
творог козельское молоко 1кг (вес)
сыр глаз вес
топленое 2,5% 1л.
сырок гл пингв понго
конфеты коробке птичье молоко село домашкино топленое 3,2%
вафли ротфронт коровка вкус сгущ 300г
круассан вар.сгущ.95г
творог первомайское молоко село домашкино 4% 0,5 л.
конфеты коробке птичье молоко 5% 200г топленое пэт
топленое бзмж 1л скоморошка
вафли вар сгущ20%40г
пирожное трубочка ваф.со
вафли вар сгущ 15% 40г премиум мол.сгущ гост
сырок твор
сыр глаз вар 

печенье ешкина коровка топленое 3,5% 1л тда
вафли голландские
напиток здоровое меню молоко паст топл. 4% бзмж
трубочка ваф.со
топленое село домашкино 4% 0,93л
печенье топленное молоко 5% 500г топленое 4,0% 930г
пряники яшкино мягкие вар. сгущ. кг
напиток овсяный ne moloko классический лайт
конфеты птичье молоко фермерское 4% 930г топленое 3,5%
сыр топленое молоко простоквашино сырок твор
сыр король артур
напиток молоко питьевое ультрапастеризованное молочный городок
молоко простоквашино сырок твор
печенье сгущеное молоко топленое пэт 930 мл 3,2%
пряники яшкино вар сгущ молоком 2кг
вафли вар.сгущ.
пряник топленое 4,0% 930г пэт 0,9л
пряник топленое 3,2% ультрапастеризованное питьевое гост 8,5% 380г
сыр глаз вар сгущ20%40г
сыр топленое м.д.ж. 4% 930 гр пэт 930мл
топленое 3,5% 1л пэт бзмж 4% 0,5 кг
напиток ne moloko овсяный
трубочка вафельная
печенье сахар сгущенное молоко 1кг (вес)
пряник тульский вар.сгущ.140г
пряники топленое вологжанка 4% пэт бзмж 1л ультрапастеризованное
топленое м.д.

In [102]:
new = pd.DataFrame()
gr =['2']*len(ext)
new['id'] = gr
new['name'] = ext
new['groups'] = gr

In [103]:
new

,id,name,groups
0,2,пряник тульский вар сгущ сах ваф ст десерт,2
1,2,топленое пюр пак гр яшкино сгущ вар молоком,2
2,2,напиток не молоко фермерское подворье бзмж суфле,2
3,2,трубочка вафельная вар сгущ молоком ип,2
4,2,пирожное трубочка со сливками,2
...,...,...,...
8995,2,напиток не молоко паст топл пак гр ооо,2
8996,2,пряники вар сгущ фл,2
8997,2,мол гост кг вес,2
8998,2,топленое вологжанка мл тфа кг вес,2


In [104]:
new.to_csv('ext9000.csv', sep=',', encoding ='utf-8', index = False)